---
**License**

 VexpBounce

 Tue Nov 15 12:13:14 2022\
 Copyright  2022\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 MagDustBounce\
 Copyright (C) 2022 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

# Magnetic field in dust dominated bouncing cosmology

In this notebook we develop the necessary objects to compute the power spectrum (and other observables) for magnetic field production in the contraction phase of a bouncing cosmology dominated by a scalar field.

The scale factor for this model is described at 1706.08830.

The electromagnetic vector field satisfy the following Hamilton equations:
\begin{equation}
\dot{A} = \frac{\Pi_A}{m}, \qquad \dot{\Pi}_A = -m\nu^2 A, \qquad m \equiv \frac{a F}{N},\quad \nu = \frac{Nk}{a},\qquad N = \frac{\mathrm{d}t}{\mathrm{d}\tau}, \qquad \alpha = \alpha_b + \frac{\tau^2}{2},\qquad \alpha = \ln(a), \qquad N = \frac{\tau}{H}.
\end{equation}
for the mode $k$. It is convenient to rewrite $F$ as
\begin{equation}
F\equiv \frac{1}{4}+B e^{-\frac{\phi^2}{\beta^2}}.
\end{equation}


## Loading NumCosmo

The first step is to load both NumCosmo and NumCosmoMath libraries.

In [ ]:
from numcosmo_py import Nc
from numcosmo_py import Ncm

import sys
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
Vexp = Nc.HICosmoVexp.new ()
Vexp.props.alphab   = 8.3163e-2
Vexp.props.sigmaphi = 9.0
Vexp.props.dphi     = -9.0e-4
Vexp.props.xb       = 2.0e36
Vexp.props.OmegaL   = 1.0
Vexp.props.Omegac   = 1.0
Vexp.props.H0       = 67.8

tc = Vexp.tau_xe (1.0e15)
reltol = 1.0e-14
B = 1.0e-58
beta = 0.08
k = 1.0

In [ ]:
tau_min = Vexp.tau_min ()
tau_max = Vexp.tau_max ()
tau_qt_c = Vexp.tau_qt_c()
tau_qt_e = Vexp.tau_qt_e()
npoints = 5000

tau_a = np.linspace (tau_min, tau_max, npoints)
tau_q_a = np.linspace (tau_qt_c, tau_qt_e, npoints)

print (f"Vexp model prepared in the interval ({tau_min},{tau_max})")
print (f"Vexp quantum dominated interval ({tau_qt_c},{tau_qt_e})")

In [ ]:
plt.figure (figsize=(14, 7))

alpha_a = [Vexp.alpha (tau) for tau in tau_a]

plt.plot (tau_a, alpha_a, label=r"$\alpha$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.show()
pass

In [ ]:
plt.figure (figsize=(14, 7))

phi_a = [Vexp.phi (tau) for tau in tau_a]

plt.plot (tau_a, phi_a, label=r"$\phi$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.show()

In [ ]:
plt.figure (figsize=(14, 7))

RS_a = [Vexp.Ricci_scale (tau) for tau in tau_a]
nu_a = [Vexp.eval_nu (tau, k) for tau in tau_a]

plt.plot (tau_a, RS_a, label=r"$R_s$")
plt.plot (tau_a, nu_a, label=r"$\nu$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.yscale('log')

plt.show()

In [ ]:
plt.figure (figsize=(14, 7))

x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_a])

plt.plot (tau_a, x_y_a[:,0], label='$x$')
plt.plot (tau_a, x_y_a[:,1], label='$y$')
plt.plot (tau_a, np.hypot(x_y_a[:,0], x_y_a[:,1]), label='$\sqrt{x^2+y^2}$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.yscale('symlog')

plt.show()

In [ ]:
#plt.figure (figsize=(14, 7))

#x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_a])

#plt.plot (tau_a, 2.0 * x_y_a[:,0]**2-1.0, label='$w = 2x^2-1$')

#plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
#leg = plt.legend (loc="best", ncol = 2, fontsize=13)

#plt.xlabel (r"$\tau$")
#plt.yscale('symlog')
#plt.ylim(-1.1, 1.1)

#plt.show()

plt.figure (figsize=(14, 7))

x_y_a = np.array ([Vexp.x_y (tau) for tau in tau_q_a])
phi_q_a = [Vexp.phi (tau) for tau in tau_q_a]
phi_q_a = np.array (phi_q_a)

plt.plot (tau_q_a, phi_q_a, label='$\phi$')
plt.plot (tau_q_a, np.exp (-(phi_q_a / 0.08)**2), label=r'$f(\phi)$')
plt.plot (tau_q_a, 2.0 * x_y_a[:,0]**2-1.0, label='$w = 2x^2-1$')

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
#plt.yscale('symlog')
plt.xlim(tau_qt_c, tau_qt_e)
plt.ylim(-2.1, 2.1)

plt.show()

In [ ]:
class PyCSQ1DMagScalarField (Ncm.CSQ1D):
  def __init__ (self, B, beta):
    Ncm.CSQ1D.__init__ (self)
    self.B = B
    self.beta = beta

  def do_eval_xi (self, model, t, k):
    return model.eval_xi (t, k, self.B, self.beta)

  def do_eval_nu (self, model, t, k):
    return model.eval_nu (t, k)

  def do_eval_nu2 (self, model, t, k):
    return model.eval_nu (t, k)**2

  def do_eval_m (self, model, t, k):
    return model.eval_m (t, self.B, self.beta)

  def do_eval_F1 (self, model, t, k):
    return model.eval_F1 (t, k, self.B, self.beta)

  def do_prepare (self, model):
    pass 

  def eval_PB_PE (self, t, c_A2 = 0.0, c_PiA2 = 0.0):
    if (c_A2 == 0.0) or (c_PiA2 == 0.0):
      (J11, J12, J22) = self.get_J_at (None, t)    
    A2   = 0.5 * J11 if c_A2   == 0.0 else c_A2
    PiA2 = 0.5 * J22 if c_PiA2 == 0.0 else c_PiA2
    F    = self.F_t (t)
    xb   = self.xb
    k    = self.get_k ()
    y    = self.y_t (t)
    RH   = self.RH
    m    = self.eval_m (None, t, k)
    PE   = PiA2   * (k / y)**3 / (2.0 * math.pi**2 * RH**4 * m)
    PB   = m * A2 * (k / y)**5 / (2.0 * math.pi**2 * RH**4)

    return PB, PE


In [ ]:
csq1d = PyCSQ1DMagScalarField (B * 1.0e60, beta * 1.0e2)
csq1d.set_k (k)

In [ ]:
print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")
#(Found2, tf) = csq1d.find_adiab_time_limit (Vexp, tau_min, tau_qt_c, 1.0e0)

In [ ]:
csq1d.eval_F1 (Vexp, -1.0, k)

In [ ]:
plt.figure (figsize=(14, 7))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

F1_a = np.array ([csq1d.eval_F1 (Vexp, tau, k) for tau in tau_ad_a])
F2_a = np.array ([csq1d.eval_F2 (Vexp, tau, k) for tau in tau_ad_a])
nu_a = np.array ([csq1d.eval_nu (Vexp, tau, k) for tau in tau_ad_a])

plt.plot (tau_ad_a, F1_a, label=r"$F_1$")
#plt.plot (tau_ad_a, F2_a, label=r"$F_2$")

plt.grid (visible=True, which='both', linestyle=':', color='0.75', linewidth=0.5)
leg = plt.legend (loc="best", ncol = 2, fontsize=13)

plt.xlabel (r"$\tau$")
plt.xscale('symlog')
plt.yscale('symlog')

plt.show()

In [ ]:
tau_min

In [ ]:
tau_i = -15.5
tau_f = 5.0
csq1d.eval_adiab_at (Vexp, tau_i)

In [ ]:
csq1d.set_init_cond_adiab (Vexp, tau_i)

In [ ]:
csq1d.set_ti (tau_i)
csq1d.set_tf (tau_f)
csq1d.prepare (Vexp)

tau_evol_a, tau_s = csq1d.get_time_array ()
tau_evol_a = np.array (tau_evol_a)

In [ ]:
y_a         = []
Abs_phi2_a  = []
Abs_Pphi2_a = []

for tau in tau_evol_a:
    (J11, J12, J22) = csq1d.get_J_at (Vexp, tau)
    Abs_phi2_a.append (0.5 * J11)
    Abs_Pphi2_a.append (0.5 * J22)

Abs_phi2_a  = np.array (Abs_phi2_a)
Abs_Pphi2_a = np.array (Abs_Pphi2_a)

In [ ]:
mylw = 1

plt.plot (tau_evol_a, np.sqrt (Abs_Pphi2_a), lw=mylw, label = f'$|\Pi_A|(k_s = {k})$')

leg = plt.legend (loc="best", ncol = 2, fontsize=13)
plt.yscale("log")

In [ ]:
mylw = 1

g0 = np.sqrt (Abs_phi2_a)[0]

plt.plot (tau_evol_a, np.abs(np.sqrt (Abs_phi2_a)/g0 - 1.0), lw=mylw, label = f'$|A_{{sk}}|(k_s = {k})$', linestyle='--')

leg = plt.legend (loc="best", ncol = 2, fontsize=13)
plt.yscale("log")